In [1]:
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objects as go
import asyncio
import pandas as pd
import numpy as np
import datetime
import stream.bogiepandas as bogiepandas
import stream.stream as stream

SERVER="nats://192.168.23.159:4222"
EXPORT_SUBJECT = "bogie"
STREAM_NAME = "test"
NUM_SENSORS = 4


In [2]:
def make_dataframes(n):

    df = pd.DataFrame()

    for i in range(n):

        x = pd.Series(ix for ix in range(10000))

        sensor_df = pd.DataFrame()
        for j in range(NUM_SENSORS):
            sensor_df["sensor%d" % j] = np.sin(x*(i+(j*10)+1))

        df2 = pd.DataFrame({"ts": [datetime.datetime.now()], "sensor_data": [sensor_df]})
        df = pd.concat([df, df2], axis=0)
    return df

all_dfs = make_dataframes(3)
#df.shape[0]

In [3]:
widgets.IntSlider(
    min=0,
    max=10,
    step=1,
    description='Slider:',
    value=3
)

start_date_picker = widgets.DatePicker(
    description='Start Date',
    disabled=False
)

slider = widgets.IntSlider(
    min=0,
    max=9,
    step=1,
    readout=False, 
)
btn_left = widgets.Button(icon="chevron-left")
btn_right = widgets.Button(icon="chevron-right")
control_box = widgets.Box([start_date_picker, btn_left, slider, btn_right])

fig = go.FigureWidget()

df = all_dfs.iloc[0]
x=list(range(0,df['sensor_data'].shape[0]))
for i in range(NUM_SENSORS):
    fig.add_trace(go.Scatter(x=[], y=[], name="sensor%d" % i))


def draw(idx):
    print("draw", idx)
    df = all_dfs.iloc[idx]
    fig.update_layout({"title": f"Dataset from {df['ts']}"})

    for i in range(NUM_SENSORS):
        fig.data[i].x = list(range(0,df['sensor_data'].shape[0]))
        fig.data[i].y = df["sensor_data"]["sensor%d" % i]



def handle_slider_change(change):
    draw(change.new)

def btn_eventhandler(obj):
    print('Hello from the {} button!'.format(obj.description))

slider.observe(handle_slider_change, names='value')
btn_right.on_click(btn_eventhandler)

vbox = widgets.VBox([control_box, fig])
display(vbox)
draw(0)


draw 0


In [7]:
ns = await stream.NatsStream.from_start_time(
    SERVER, STREAM_NAME, EXPORT_SUBJECT, "1990-01-01T00:00:00.000000Z"
)

while True:
    msg = await ns.next_msg()
    if msg is None:
        break
    #print(msg.metadata.timestamp)
    df = bogiepandas.bogie_nats_to_pandas(msg)
    print(df)
    print(df["sensor_data"][0])


                nats_rx_time               trigger_time  \
0 2022-08-29 14:37:33.379134 2022-08-29 12:37:33.376664   

                                         sensor_data  
0         sensor0   sensor1   sensor2   sensor3
0...  
       sensor0   sensor1   sensor2   sensor3
0     0.584062  0.593438  0.592188  0.598125
1     0.584062  0.593438  0.592188  0.597813
2     0.584062  0.593438  0.592500  0.598125
3     0.584375  0.593438  0.592188  0.597813
4     0.584375  0.593438  0.592188  0.597813
...        ...       ...       ...       ...
3995  0.584375  0.593438  0.592500  0.598125
3996  0.584062  0.593438  0.592188  0.598125
3997  0.584062  0.593750  0.592188  0.597813
3998  0.584062  0.593438  0.592500  0.597813
3999  0.584375  0.593750  0.592188  0.598125

[4000 rows x 4 columns]
                nats_rx_time               trigger_time  \
0 2022-08-29 14:37:42.359588 2022-08-29 12:37:42.357753   

                                         sensor_data  
0         sensor0   sensor1   se